In [4]:
import kagglehub
import pyro
import pyro.distributions as dist
import pyGMs as gm
import torch
import pandas as pd
import os

# Download latest version
path = kagglehub.dataset_download("drgilermo/nba-players-stats")
print("Path to dataset files:", path)

100%|█████████████████████████████████████████████████████████████████████████████| 2.13M/2.13M [00:01<00:00, 2.21MB/s]

Extracting files...
Path to dataset files: C:\Users\chawk\.cache\kagglehub\datasets\drgilermo\nba-players-stats\versions\2


In [8]:
df = pd.read_csv(os.path.join(path, "Seasons_Stats.csv"), encoding='latin1')
df = df.drop(columns=["Unnamed: 0"])
df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,0.368,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,0.435,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,0.394,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,0.312,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,0.308,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0


In [14]:
mvp_winners = {
    1955: "Bob Pettit",
    1956: "Bob Cousy",
    1957: "Bill Russell",
    1958: "Bob Pettit",
    1959: "Wilt Chamberlain",
    1960: "Bill Russell",
    1961: "Bill Russell",
    1962: "Bill Russell",
    1963: "Oscar Robertson",
    1964: "Bill Russell",
    1965: "Wilt Chamberlain",
    1966: "Wilt Chamberlain",
    1967: "Wilt Chamberlain",
    1968: "Wes Unseld",
    1969: "Willis Reed",
    1970: "Kareem Abdul-Jabbar",
    1971: "Kareem Abdul-Jabbar",
    1972: "Dave Cowens",
    1973: "Kareem Abdul-Jabbar",
    1974: "Bob McAdoo",
    1975: "Kareem Abdul-Jabbar",
    1976: "Kareem Abdul-Jabbar",
    1977: "Bill Walton",
    1978: "Moses Malone",
    1979: "Kareem Abdul-Jabbar",
    1980: "Julius Erving",
    1981: "Moses Malone",
    1982: "Moses Malone",
    1983: "Larry Bird",
    1984: "Larry Bird",
    1985: "Larry Bird",
    1986: "Magic Johnson",
    1987: "Michael Jordan",
    1988: "Magic Johnson",
    1989: "Magic Johnson",
    1990: "Michael Jordan",
    1991: "Michael Jordan",
    1992: "Charles Barkley",
    1993: "Hakeem Olajuwon",
    1994: "David Robinson",
    1995: "Michael Jordan",
    1996: "Karl Malone",
    1997: "Michael Jordan",
    1998: "Karl Malone",
    1999: "Shaquille O'Neal",
    2000: "Allen Iverson",
    2001: "Tim Duncan",
    2002: "Tim Duncan",
    2003: "Kevin Garnett",
    2004: "Steve Nash",
    2005: "Steve Nash",
    2006: "Dirk Nowitzki",
    2007: "Kobe Bryant",
    2008: "LeBron James",
    2009: "LeBron James",
    2010: "Derrick Rose",
    2011: "LeBron James",
    2012: "LeBron James",
    2013: "Kevin Durant",
    2014: "Stephen Curry",
    2015: "Stephen Curry",
    2016: "Russell Westbrook",
    2017: "James Harden"
}

In [17]:
import random
random.seed(42)

years = list(mvp_winners.keys())
random.shuffle(years)
split_index = int(0.8 * len(years))
train_years = years[:split_index]
test_years = years[split_index:]

train_mvp_winners = {year: mvp_winners[year] for year in train_years}
test_mvp_winners = {year: mvp_winners[year] for year in test_years}

print(train_mvp_winners)
print(test_mvp_winners)

{1991: 'Michael Jordan', 1976: 'Kareem Abdul-Jabbar', 1996: 'Karl Malone', 1988: 'Magic Johnson', 1974: 'Bob McAdoo', 2017: 'James Harden', 2012: 'LeBron James', 2006: 'Dirk Nowitzki', 1994: 'David Robinson', 1985: 'Larry Bird', 2016: 'Russell Westbrook', 1986: 'Magic Johnson', 1973: 'Kareem Abdul-Jabbar', 1978: 'Moses Malone', 1964: 'Bill Russell', 1971: 'Kareem Abdul-Jabbar', 1966: 'Wilt Chamberlain', 1975: 'Kareem Abdul-Jabbar', 2003: 'Kevin Garnett', 2001: 'Tim Duncan', 1958: 'Bob Pettit', 1984: 'Larry Bird', 2008: 'LeBron James', 1959: 'Wilt Chamberlain', 2010: 'Derrick Rose', 1965: 'Wilt Chamberlain', 2007: 'Kobe Bryant', 1977: 'Bill Walton', 1999: "Shaquille O'Neal", 1980: 'Julius Erving', 1979: 'Kareem Abdul-Jabbar', 1955: 'Bob Pettit', 2013: 'Kevin Durant', 1983: 'Larry Bird', 1997: 'Michael Jordan', 1981: 'Moses Malone', 2005: 'Steve Nash', 1967: 'Wilt Chamberlain', 1990: 'Michael Jordan', 2000: 'Allen Iverson', 1993: 'Hakeem Olajuwon', 1987: 'Michael Jordan', 2011: 'LeBron J

In [10]:
df['MVP_winner'] = df['year'].map(mvp_winners)
df['Won_MVP?'] = df['PLAYER'] == df['MVP_winner']
df.head()

KeyError: 'year'

In [48]:
print(df[df['Won_MVP?'] == True])

         year     Season_type  PLAYER_ID  RANK                 PLAYER  \
2     2012-13  Regular_Season       2544     3           LeBron James   
468   2013-14  Regular_Season     201142     1           Kevin Durant   
951   2014-15  Regular_Season     201939     2          Stephen Curry   
1443  2015-16  Regular_Season     201939     2          Stephen Curry   
1918  2016-17  Regular_Season     201566     1      Russell Westbrook   
2405  2017-18  Regular_Season     201935     2           James Harden   
2950  2018-19  Regular_Season     203507     7  Giannis Antetokounmpo   
3477  2019-20  Regular_Season     203507     4  Giannis Antetokounmpo   
4005  2020-21  Regular_Season     203999     3           Nikola Jokic   
4547  2021-22  Regular_Season     203999     5           Nikola Jokic   
5149  2022-23  Regular_Season     203954     2            Joel Embiid   
5691  2023-24  Regular_Season     203999     5           Nikola Jokic   

         TEAM_ID TEAM  GP   MIN  FGM  ...  AST  ST